In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

from models import *

global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
batch_size = 128
model_name = "VGG16_squeezed"
model = VGG16_squeezed()
print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [150, 225]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
VGG_quant(
  (features): Sequential(
    (0): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d(
      128, 128, kernel_size=(3, 3), stride

In [ ]:
import torch.optim.lr_scheduler as lr_scheduler

# --- 1. Set model save directory ---
fdir = 'result/VGG16_quant'
os.makedirs(fdir, exist_ok=True) # Ensure directory exists

# --- 2. Move model to GPU and define loss function ---
model = model.cuda()
criterion = nn.CrossEntropyLoss().cuda()

# --- 3. Define training epochs and optimizer ---
num_epochs = 300
# Use AdamW optimizer for faster convergence
optimizer = optim.AdamW(model.parameters(), lr=5e-3, weight_decay=1e-2) 

# --- 4. Define learning rate scheduler ---
# Use Cosine Annealing, T_max must equal total number of epochs
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

# --- 5. Start training loop ---
best_prec = 0
print(f"==> Start training model for {num_epochs} epochs...")

for epoch in range(num_epochs):
    
    # Train for one epoch
    train(trainloader, model, criterion, optimizer, epoch)
    
    # Validate model
    prec = validate(testloader, model, criterion)
    
    # Call scheduler.step() after validate and before saving checkpoint
    scheduler.step()
    
    # Check if it is the best model
    is_best = prec > best_prec
    best_prec = max(prec, best_prec)
    
    # Save checkpoint
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)
    
    # Print current epoch learning rate and precision
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch {epoch+1}/{num_epochs} Completed | Learning Rate: {current_lr:.6f} | Val Precision: {prec:.2f}% | Best Precision: {best_prec:.2f}%")

print(f"==> Training completed. Best model saved to {fdir}/model_best.pth.tar")

In [2]:
PATH = "result/VGG16_quant/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))

C:\Users\ASUS\AppData\Local\Temp\ipykernel_11556\564645170.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(PATH)



Test set: Accuracy: 9025/10000 (90%)



In [3]:
# --- 1. Locate the target layer ---
print("Searching for the index of the 16x16 convolution layer...")
target_layer_idx = -1
for i, layer in enumerate(model.features):
    if isinstance(layer, torch.nn.Conv2d):
        if layer.in_channels == 16 and layer.out_channels == 16:
            target_layer_idx = i
            print(f"✅ Target layer found! Index = {i}")
            break

Searching for the index of the 16x16 convolution layer...
✅ Target layer found! Index = 34


In [4]:
# --- 2. Capture and Extract Quantized Data (Hook & Extract) ---

# A. Prepare Hook
inputs_q = []
def hook(module, input, output):
    # input[0] is the Activation we want
    inputs_q.append(input[0].detach().cpu())

# B. Register Hook to the layer found earlier
target_layer = model.features[target_layer_idx] # Use the index found earlier
handle = target_layer.register_forward_hook(hook)

# C. Run one forward pass (Inference)
# Take one Batch from the test loader
data, _ = next(iter(testloader)) 
data = data.cuda()
model(data) # Trigger Hook

# D. Remove Hook
handle.remove()

# E. Process extracted data
# 1. Process Activation (2-bit)
act = inputs_q[0] # Retrieve input of the first Batch
act_alpha = target_layer.act_alpha.detach().cpu()
act_bit = 2 # [Key] Force set to 2-bit
# Quantization formula: float -> int
act_q = model.features[target_layer_idx].act_alq(act.cuda(), act_alpha.cuda()).cpu()
act_int = act_q / (act_alpha / (2**act_bit - 1))
act_int = act_int.round() # Ensure it is an integer 0, 1, 2, 3

# 2. Process Weight (4-bit) - Revised Version
# Directly get the quantized weights (Fake Quantized Float) calculated and stored during forward
# Note: This requires having run model(data) earlier, because self.weight_q is updated in forward
weight_fake_quant = target_layer.weight_q.detach().cpu() 

wgt_alpha = target_layer.weight_quant.wgt_alpha.detach().cpu()
w_bit = 4

# Derive Integer: Integer = FakeFloat / Scale
# Scale = alpha / (2^(b-1) - 1)
scale = wgt_alpha / (2**(w_bit-1) - 1)

# Divide by scale directly to get integer
weight_int = weight_fake_quant / scale

# Round to ensure it is an integer (eliminate floating point error)
weight_int = weight_int.round()

# Clamp between -7 and 7 (4-bit signed range: -8 to 7, but symmetric quantization is usually -7 to 7)
# Check uniform_quant implementation in quant_layer.py, it uses clamp(min=-1, max=1)
# So integer range should be -(2^(b-1)-1) to +(2^(b-1)-1), i.e., -7 to +7
weight_int = weight_int.clamp(-(2**(w_bit-1)-1), (2**(w_bit-1)-1))

print("Data extraction completed!")
print(f"Activation Shape: {act_int.shape}")
print(f"Weight Shape: {weight_int.shape}")

Data extraction completed!
Activation Shape: torch.Size([128, 16, 2, 2])
Weight Shape: torch.Size([16, 16, 3, 3])


In [5]:

print("🚀 Generating V3 Hardware Test Data (Using Real Extracted Data)...")

# --- 1. Data Preparation and Slicing ---
# Ensure data dimensions match requirements: IC=16, OC=16
# act_int original shape might be [1, 64, H, W], weight_int is [64, 64, 3, 3]
# We force slice to take the first 16 channels

# [Batch, IC, H, W] -> Take Batch 0, IC 0-16
real_act = act_int[0:1, 0:16, :, :].float() 
# [OC, IC, K, K] -> Take OC 0-16, IC 0-16
real_weight = weight_int[0:16, 0:16, :, :].float()

print(f"Activation Shape Used: {real_act.shape}")
print(f"Weight Shape Used: {real_weight.shape}")

# Padding to 4x4 (VGG usually has padding=1, 2x2 input becomes 4x4 after padding)
act_padded = F.pad(real_act, (1, 1, 1, 1), "constant", 0) 
# Flatten: [1, 16, 4, 4] -> [1, 16, 16] (Time step)
act_flat = act_padded.view(1, 16, -1) 

def decimal_to_hex(val, width=4):
    val = int(val)
    if val < 0: val = (1 << width) + val
    return f'{val & ((1<<width)-1):01x}'

# --- 2. Activation Packing (16 lines) ---
# The logic here is correct: it packs data from 16 Input Channels within one time step
# r=0 -> IC 0,1; r=7 -> IC 14,15
print(f" -> Writing activationSIMD.txt ...")
with open('activationSIMD.txt', 'w') as f:
    for t in range(16): # 16 time steps (Pixels)
        line_hex = ""
        for r in range(7, -1, -1): # 8 Rows, each Row contains 2 Channels
            ch_low = 2 * r
            ch_high = 2 * r + 1
            
            # Get value from real data
            val_low = act_flat[0, ch_low, t].item()
            val_high = act_flat[0, ch_high, t].item()
            
            # 2-bit Packing
            packed_val = (int(val_high) << 2) | (int(val_low) & 0b11)
            line_hex += decimal_to_hex(packed_val, 4)
        f.write(line_hex + '\n')

# --- 3. Weight Packing (36 lines - Only IC 0 & 1) ---
print(f" -> Writing weightSIMD.txt (Generating weights for IC 0 & 1 only)...")
with open('weightSIMD.txt', 'w') as f:
    for tile in range(2): # 2 Output Channel Tiles (0-7, 8-15)
        start_oc = tile * 8
        for ky in range(3):
            for kx in range(3):
                # Low 4-bits: Weights corresponding to IC 0
                line_low = ""
                for oc_idx in range(7, -1, -1):
                    out_c = start_oc + oc_idx
                    # Get real weights (IC=0)
                    w_val = real_weight[out_c, 0, ky, kx].item() 
                    line_low += decimal_to_hex(w_val, 4)
                f.write(line_low + '\n')
                
                # High 4-bits: Weights corresponding to IC 1
                line_high = ""
                for oc_idx in range(7, -1, -1):
                    out_c = start_oc + oc_idx
                    # Get real weights (IC=1)
                    w_val = real_weight[out_c, 1, ky, kx].item()
                    line_high += decimal_to_hex(w_val, 4)
                f.write(line_high + '\n')

# --- 4. Output Ref (Revised Version) ---
# Critical Fix: To match hardware behavior, when calculating the Reference here,
# we must simulate the hardware scenario where "only IC 0 and IC 1 weights are loaded".
# So we manually set weights for IC 2 to 15 to zero, then perform convolution.

weight_simulated = real_weight.clone()
weight_simulated[:, 2:, :, :] = 0 # [Key] Set IC 2-15 weights to 0 because they were not written to the file

print(" -> Calculating Reference Output (Simulating Partial Execution)...")
# Execute Convolution + ReLU
out_ref = F.relu(F.conv2d(act_padded, weight_simulated, padding=0))

with open('output_ref_SIMD.txt', 'w') as f:
    H_out, W_out = out_ref.shape[2], out_ref.shape[3]
    for h in range(H_out):
        for w in range(W_out):
            for tile in range(2):
                start_oc = tile * 8
                line = ""
                for oc_idx in range(7, -1, -1):
                    val = int(out_ref[0, start_oc+oc_idx, h, w].item())
                    line += decimal_to_hex(val, 4)
                f.write(line + '\n')

# Rewrite output writer logic just in case (adapt for psum_bw=16)
def decimal_to_hex_safe(val, bits=16):
    val = int(val)
    if val < 0: val = (1 << bits) + val
    return f'{val & ((1<<bits)-1):04x}' # 4 hex chars for 16 bits

with open('output_ref_SIMD.txt', 'w') as f:
    H_out, W_out = out_ref.shape[2], out_ref.shape[3]
    for h in range(H_out):
        for w in range(W_out):
            for tile in range(2):
                start_oc = tile * 8
                line = ""
                for oc_idx in range(7, -1, -1):
                    val = int(out_ref[0, start_oc+oc_idx, h, w].item())
                    line += decimal_to_hex_safe(val, 16) # Use 16-bit width
                f.write(line + '\n')

print("✅ Done! Real data generated, and Reference calibrated for 36-line weight truncation.")

🚀 Generating V3 Hardware Test Data (Using Real Extracted Data)...
Activation Shape Used: torch.Size([1, 16, 2, 2])
Weight Shape Used: torch.Size([16, 16, 3, 3])
 -> Writing activationSIMD.txt ...
 -> Writing weightSIMD.txt (Generating weights for IC 0 & 1 only)...
 -> Calculating Reference Output (Simulating Partial Execution)...
✅ Done! Real data generated, and Reference calibrated for 36-line weight truncation.
